In [57]:
from Bio import SeqIO
import pandas as pd
import numpy as np


# chrdict = {}

# for i, record in enumerate(SeqIO.parse("..\\dataset\\\GRCh37.primary_assembly.genome.fa", "fasta")):
#     chrdict[record.id] = record.seq


# LE SE SEQUENZE DI BASI AZOTATE SONO MEMORIZZATE IN UN DIZIONARIO CHE USA COME CHIAVE "chr#" dove # indica
# il numero del cromosoma oppure X/Y/M (M non ci interessa perché é il DNA mitocondriale che non viene usato per questo caso)

In [8]:
chrdict

{'chr1': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr10': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr11': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr12': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr13': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr14': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr15': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr16': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr17': Seq('AAGCTTCTCACCCTGTTCCTGCATAGATAATTGCATGACAATTGCCTTGTCCCT...GGT'),
 'chr18': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr19': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr2': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr20': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

In [8]:
# UA INIZIA INVECE LA PARTE DI LETTURA DEL FILE CONTENENTE I TSS

from Bio import SeqIO
import pandas as pd
import numpy as np
def extract_p(p):
    return p.split("@")[0]

def extract_gene_name(string):
    string =  string.split("@")[1]
    return string.split(",")[0]

v1 = np.vectorize(extract_p)
v2 = np.vectorize(extract_gene_name)

df = pd.read_csv("..\\dataset\\TSS_human.bed", delimiter="\t", skiprows=1, names=["chromosome_name", "start", "end", "locus_id", "color", "strand", "start2", "end2", "RGB"])
df




,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB
0,chr10,100008587,100008589,"p1@CU680531,0.1352",-89.0,+,100008587,100008589,"211,211,211"
1,chr10,100015362,100015397,"p2@LOXL4,0.1291",55.0,-,100015362,100015397,"211,211,211"
2,chr10,100017518,100017519,"p3@LOXL4,0.1842",13.0,-,100017518,100017519,"30,144,255"
3,chr10,100027943,100027958,"p1@LOXL4,0.2200",48.0,-,100027943,100027958,"30,144,255"
4,chr10,100174900,100174956,"p1@PYROXD2,0.2721",0.0,-,100174900,100174956,"60,179,113"
...,...,...,...,...,...,...,...,...,...
1048119,chrY,9958814,9958815,"p@chrY:9958814..9958815,-,0.0551",0.0,-,9958814,9958815,"211,211,211"
1048120,chrY,9959226,9959229,"p@chrY:9959226..9959229,-,0.0639",0.0,-,9959226,9959229,"211,211,211"
1048121,chrY,9968102,9968103,"p@chrY:9968102..9968103,-,0.1091",0.0,-,9968102,9968103,"211,211,211"
1048122,chrY,9985705,9985711,"p@chrY:9985705..9985711,-,0.0853",0.0,-,9985705,9985711,"211,211,211"


In [4]:
# Applica la funzione alla colonna del DataFrame
df['p'] = v1(df['locus_id'].values)
df['gene_name'] = v2(df['locus_id'].values)
df = df.drop(columns=["locus_id", "color", "start2", "end2", "RGB"])

df

,chromosome_name,start,end,strand,p,gene_name
0,chr10,100008587,100008589,+,p1,CU680531
1,chr10,100015362,100015397,-,p2,LOXL4
2,chr10,100017518,100017519,-,p3,LOXL4
3,chr10,100027943,100027958,-,p1,LOXL4
4,chr10,100174900,100174956,-,p1,PYROXD2
...,...,...,...,...,...,...
1048119,chrY,9958814,9958815,-,p,chrY:9958814..9958815
1048120,chrY,9959226,9959229,-,p,chrY:9959226..9959229
1048121,chrY,9968102,9968103,-,p,chrY:9968102..9968103
1048122,chrY,9985705,9985711,-,p,chrY:9985705..9985711


In [5]:
import re

def estrai_numero(s):
    match = re.match(r'p(\d+)', s)
    if match:
        return int(match.group(1))
    else:
        return None


df['p_value'] = df['p'].apply(estrai_numero)
#droppa righe con p_value non definito
df = df.dropna()
df

,chromosome_name,start,end,strand,p,gene_name,p_value
0,chr10,100008587,100008589,+,p1,CU680531,1.0
1,chr10,100015362,100015397,-,p2,LOXL4,2.0
2,chr10,100017518,100017519,-,p3,LOXL4,3.0
3,chr10,100027943,100027958,-,p1,LOXL4,1.0
4,chr10,100174900,100174956,-,p1,PYROXD2,1.0
...,...,...,...,...,...,...,...
294760,chrY,7142165,7142189,+,p3,PRKY,3.0
294761,chrY,7142206,7142215,+,p9,PRKY,9.0
294762,chrY,7200834,7200838,+,p12,PRKY,12.0
294763,chrY,7200913,7200914,+,p14,PRKY,14.0


In [6]:
df['p_value'].value_counts()

p_value
1.0      39445
2.0      28101
3.0      23053
4.0      19577
5.0      16960
         ...  
213.0        1
218.0        1
207.0        1
209.0        1
219.0        1
Name: count, Length: 282, dtype: int64

In [14]:
df.groupby("gene_name")["p_value"].idxmin()


gene_name
A1BG           65039
A1BG-AS1       65040
A1CF            4033
A2LD1          27660
A2M            27124
               ...  
uc011mav.1    287241
uc011mct.1    159387
uc011mds.1    288056
uc011mfh.1    290683
uc011mfi.1    290713
Name: p_value, Length: 39802, dtype: int64

In [11]:
df['p_value'].value_counts()

p_value
1.0     39445
2.0       281
3.0        21
4.0        19
5.0         8
7.0         7
6.0         4
12.0        4
10.0        2
9.0         2
19.0        2
8.0         2
15.0        1
21.0        1
22.0        1
13.0        1
11.0        1
Name: count, dtype: int64

In [8]:
df = df.loc[df.groupby("gene_name")["p_value"].idxmin()]
df

,chromosome_name,start,end,strand,p,gene_name,p_value
65039,chr19,58858938,58859039,-,p1,A1BG,1.0
65040,chr19,58859101,58859149,+,p1,A1BG-AS1,1.0
4033,chr10,52645379,52645393,-,p1,A1CF,1.0
27660,chr13,101240987,101241049,-,p1,A2LD1,1.0
27124,chr12,9268507,9268523,-,p1,A2M,1.0
...,...,...,...,...,...,...,...
287241,chr9,131012693,131012727,-,p1,uc011mav.1,1.0
159387,chr9,135895676,135895723,+,p1,uc011mct.1,1.0
288056,chr9,138899746,138899753,-,p1,uc011mds.1,1.0
290683,chrM,6330,6392,+,p1,uc011mfh.1,1.0


In [9]:
df['p_value'].value_counts()


p_value
1.0     39445
2.0       281
3.0        21
4.0        19
5.0         8
7.0         7
6.0         4
12.0        4
10.0        2
9.0         2
19.0        2
8.0         2
15.0        1
21.0        1
22.0        1
13.0        1
11.0        1
Name: count, dtype: int64

In [ ]:
df.drop(columns=["p", "p_value"], inplace=True)
df

In [18]:
df['chromosome_name'].value_counts()

chromosome_name
chr1     3852
chr2     2840
chr11    2230
chr3     2212
chr12    2174
chr19    2142
chr7     2137
chr6     2121
chr17    2091
chr5     2015
chr14    1772
chr4     1683
chr8     1621
chr10    1562
chr9     1488
chrX     1374
chr16    1358
chr15    1326
chr20    1045
chr22     946
chr13     748
chr18     512
chr21     463
chrY       53
chrM       37
Name: count, dtype: int64

In [13]:
df

,chromosome_name,start,end,strand,p,gene_name,p_value
65039,chr19,58858938,58859039,-,p1,A1BG,1.0
65040,chr19,58859101,58859149,+,p1,A1BG-AS1,1.0
4033,chr10,52645379,52645393,-,p1,A1CF,1.0
27660,chr13,101240987,101241049,-,p1,A2LD1,1.0
27124,chr12,9268507,9268523,-,p1,A2M,1.0
...,...,...,...,...,...,...,...
287241,chr9,131012693,131012727,-,p1,uc011mav.1,1.0
159387,chr9,135895676,135895723,+,p1,uc011mct.1,1.0
288056,chr9,138899746,138899753,-,p1,uc011mds.1,1.0
290683,chrM,6330,6392,+,p1,uc011mfh.1,1.0


In [14]:
# rename column start in TSS
df.rename(columns={"start":"TSS"}, inplace=True)
df

,chromosome_name,TSS,end,strand,p,gene_name,p_value
65039,chr19,58858938,58859039,-,p1,A1BG,1.0
65040,chr19,58859101,58859149,+,p1,A1BG-AS1,1.0
4033,chr10,52645379,52645393,-,p1,A1CF,1.0
27660,chr13,101240987,101241049,-,p1,A2LD1,1.0
27124,chr12,9268507,9268523,-,p1,A2M,1.0
...,...,...,...,...,...,...,...
287241,chr9,131012693,131012727,-,p1,uc011mav.1,1.0
159387,chr9,135895676,135895723,+,p1,uc011mct.1,1.0
288056,chr9,138899746,138899753,-,p1,uc011mds.1,1.0
290683,chrM,6330,6392,+,p1,uc011mfh.1,1.0


In [19]:
df['TSS'].unique().shape
#trova i tss duplicati
df[df.duplicated(subset="TSS", keep=False)].sort_values(by="TSS")

,chromosome_name,TSS,end,strand,p,gene_name,p_value
26214,chr12,7283822,7283827,-,p1,ENST00000538062,1.0
279440,chr7,7283822,7283840,+,p1,uc003srb.1,1.0
47188,chr16,81272287,81272303,+,p1,BCMO1,1.0
5538,chr10,81272287,81272299,+,p1,EIF5AL1,1.0
37357,chr15,101459519,101459539,-,p2,ENST00000560351,2.0
37358,chr15,101459519,101459585,+,p1,LRRK1,1.0


In [12]:
lizio=pd.read_csv("..\\dataset\\Join_with_Lizio.csv",sep=",")
lizio.drop(columns=["Unnamed: 0", ],inplace=True)
lizio

,gene_name,gene_stable_id,gene_id,gene_type,TSS,strand,chromosome_name
0,A1BG,ENSG00000121410,ENSG00000121410.12,protein_coding,58347673,-,chr19
1,A1CF,ENSG00000148584,ENSG00000148584.15,protein_coding,50885633,-,chr10
2,A2M,ENSG00000175899,ENSG00000175899.15,protein_coding,9115927,-,chr12
3,A2ML1,ENSG00000166535,ENSG00000166535.20,protein_coding,8822627,+,chr12
4,A4GALT,ENSG00000128274,ENSG00000128274.17,protein_coding,42720879,-,chr22
...,...,...,...,...,...,...,...
15615,ZYG11A,ENSG00000203995,ENSG00000203995.10,protein_coding,52842770,+,chr1
15616,ZYG11B,ENSG00000162378,ENSG00000162378.13,protein_coding,52726478,+,chr1
15617,ZYX,ENSG00000159840,ENSG00000159840.16,protein_coding,143381361,+,chr7
15618,ZZEF1,ENSG00000074755,ENSG00000074755.15,protein_coding,4143055,-,chr17


# join with lizio e file bed con TSS
cosi trovo i TSS dei geni per la build 37 usando i tss de file bed

In [20]:
# join with lizio e file bed con TSS
tss_merge = pd.merge(df, lizio, on="TSS")
tss_merge

,chromosome_name_x,TSS,end,strand_x,p,gene_name_x,p_value,gene_name_y,gene_stable_id,gene_id,gene_type,strand_y,chromosome_name_y
0,chrX,112285920,112285928,+,p1,AK127862,1.0,TIFA,ENSG00000145365,ENSG00000145365.11,protein_coding,-,chr4
1,chr17,43025265,43025277,+,p1,BC031668,1.0,RND2,ENSG00000108830,ENSG00000108830.10,protein_coding,+,chr17
2,chr3,73232465,73232490,-,p1,CCDC75P1,1.0,C17orf80,ENSG00000141219,ENSG00000141219.16,protein_coding,+,chr17
3,chr5,43397184,43397286,-,p1,CCL28,1.0,CCL28,ENSG00000151882,ENSG00000151882.11,protein_coding,-,chr5
4,chr5,1594567,1594622,-,p1,SDHAP3,1.0,FBXL14,ENSG00000171823,ENSG00000171823.7,protein_coding,-,chr12


## merge del bed con lizio per prendere i geni uguale a quelli di lizio

In [60]:
gene_name_merge = pd.merge(df, lizio, on="gene_name")
#prendi gene_name dove strand_x e strand_y sono uguali
gene_name_merge

,chromosome_name_x,TSS_x,end,strand_x,p,gene_name,p_value,gene_stable_id,gene_id,gene_type,TSS_y,strand_y,chromosome_name_y
0,chr19,58858938,58859039,-,p1,A1BG,1.0,ENSG00000121410,ENSG00000121410.12,protein_coding,58347673,-,chr19
1,chr10,52645379,52645393,-,p1,A1CF,1.0,ENSG00000148584,ENSG00000148584.15,protein_coding,50885633,-,chr10
2,chr12,9268507,9268523,-,p1,A2M,1.0,ENSG00000175899,ENSG00000175899.15,protein_coding,9115927,-,chr12
3,chr12,8975206,8975223,+,p1,A2ML1,1.0,ENSG00000166535,ENSG00000166535.20,protein_coding,8822627,+,chr12
4,chr22,43116867,43116885,-,p1,A4GALT,1.0,ENSG00000128274,ENSG00000128274.17,protein_coding,42720879,-,chr22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15615,chr1,53308425,53308442,+,p1,ZYG11A,1.0,ENSG00000203995,ENSG00000203995.10,protein_coding,52842770,+,chr1
15616,chr1,53192114,53192150,+,p1,ZYG11B,1.0,ENSG00000162378,ENSG00000162378.13,protein_coding,52726478,+,chr1
15617,chr7,143078379,143078454,+,p1,ZYX,1.0,ENSG00000159840,ENSG00000159840.16,protein_coding,143381361,+,chr7
15618,chr17,4046257,4046349,-,p1,ZZEF1,1.0,ENSG00000074755,ENSG00000074755.15,protein_coding,4143055,-,chr17


In [61]:
gene_name_merge = gene_name_merge[gene_name_merge["strand_x"]==gene_name_merge["strand_y"]]

gene_name_merge

,chromosome_name_x,TSS_x,end,strand_x,p,gene_name,p_value,gene_stable_id,gene_id,gene_type,TSS_y,strand_y,chromosome_name_y
0,chr19,58858938,58859039,-,p1,A1BG,1.0,ENSG00000121410,ENSG00000121410.12,protein_coding,58347673,-,chr19
1,chr10,52645379,52645393,-,p1,A1CF,1.0,ENSG00000148584,ENSG00000148584.15,protein_coding,50885633,-,chr10
2,chr12,9268507,9268523,-,p1,A2M,1.0,ENSG00000175899,ENSG00000175899.15,protein_coding,9115927,-,chr12
3,chr12,8975206,8975223,+,p1,A2ML1,1.0,ENSG00000166535,ENSG00000166535.20,protein_coding,8822627,+,chr12
4,chr22,43116867,43116885,-,p1,A4GALT,1.0,ENSG00000128274,ENSG00000128274.17,protein_coding,42720879,-,chr22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15615,chr1,53308425,53308442,+,p1,ZYG11A,1.0,ENSG00000203995,ENSG00000203995.10,protein_coding,52842770,+,chr1
15616,chr1,53192114,53192150,+,p1,ZYG11B,1.0,ENSG00000162378,ENSG00000162378.13,protein_coding,52726478,+,chr1
15617,chr7,143078379,143078454,+,p1,ZYX,1.0,ENSG00000159840,ENSG00000159840.16,protein_coding,143381361,+,chr7
15618,chr17,4046257,4046349,-,p1,ZZEF1,1.0,ENSG00000074755,ENSG00000074755.15,protein_coding,4143055,-,chr17


In [62]:
gene_name_merge = gene_name_merge[gene_name_merge["chromosome_name_x"]==gene_name_merge["chromosome_name_y"]]
gene_name_merge

,chromosome_name_x,TSS_x,end,strand_x,p,gene_name,p_value,gene_stable_id,gene_id,gene_type,TSS_y,strand_y,chromosome_name_y
0,chr19,58858938,58859039,-,p1,A1BG,1.0,ENSG00000121410,ENSG00000121410.12,protein_coding,58347673,-,chr19
1,chr10,52645379,52645393,-,p1,A1CF,1.0,ENSG00000148584,ENSG00000148584.15,protein_coding,50885633,-,chr10
2,chr12,9268507,9268523,-,p1,A2M,1.0,ENSG00000175899,ENSG00000175899.15,protein_coding,9115927,-,chr12
3,chr12,8975206,8975223,+,p1,A2ML1,1.0,ENSG00000166535,ENSG00000166535.20,protein_coding,8822627,+,chr12
4,chr22,43116867,43116885,-,p1,A4GALT,1.0,ENSG00000128274,ENSG00000128274.17,protein_coding,42720879,-,chr22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15615,chr1,53308425,53308442,+,p1,ZYG11A,1.0,ENSG00000203995,ENSG00000203995.10,protein_coding,52842770,+,chr1
15616,chr1,53192114,53192150,+,p1,ZYG11B,1.0,ENSG00000162378,ENSG00000162378.13,protein_coding,52726478,+,chr1
15617,chr7,143078379,143078454,+,p1,ZYX,1.0,ENSG00000159840,ENSG00000159840.16,protein_coding,143381361,+,chr7
15618,chr17,4046257,4046349,-,p1,ZZEF1,1.0,ENSG00000074755,ENSG00000074755.15,protein_coding,4143055,-,chr17


In [63]:
#drop tutte le colonne che finiscono con _y
gene_name_merge = gene_name_merge.loc[:,~gene_name_merge.columns.str.endswith("_y")]
gene_name_merge

,chromosome_name_x,TSS_x,end,strand_x,p,gene_name,p_value,gene_stable_id,gene_id,gene_type
0,chr19,58858938,58859039,-,p1,A1BG,1.0,ENSG00000121410,ENSG00000121410.12,protein_coding
1,chr10,52645379,52645393,-,p1,A1CF,1.0,ENSG00000148584,ENSG00000148584.15,protein_coding
2,chr12,9268507,9268523,-,p1,A2M,1.0,ENSG00000175899,ENSG00000175899.15,protein_coding
3,chr12,8975206,8975223,+,p1,A2ML1,1.0,ENSG00000166535,ENSG00000166535.20,protein_coding
4,chr22,43116867,43116885,-,p1,A4GALT,1.0,ENSG00000128274,ENSG00000128274.17,protein_coding
...,...,...,...,...,...,...,...,...,...,...
15615,chr1,53308425,53308442,+,p1,ZYG11A,1.0,ENSG00000203995,ENSG00000203995.10,protein_coding
15616,chr1,53192114,53192150,+,p1,ZYG11B,1.0,ENSG00000162378,ENSG00000162378.13,protein_coding
15617,chr7,143078379,143078454,+,p1,ZYX,1.0,ENSG00000159840,ENSG00000159840.16,protein_coding
15618,chr17,4046257,4046349,-,p1,ZZEF1,1.0,ENSG00000074755,ENSG00000074755.15,protein_coding


In [64]:
#rinomina le colonne che finiscono con _x togliendo il suffisso
gene_name_merge.rename(columns={"chromosome_name_x":"chromosome_name", "strand_x":"strand", "TSS_x": "TSS"}, inplace=True)

C:\Users\39379\AppData\Local\Temp\ipykernel_7732\725091559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_name_merge.rename(columns={"chromosome_name_x":"chromosome_name", "strand_x":"strand", "TSS_x": "TSS"}, inplace=True)


In [65]:
gene_name_merge.drop(columns=["p_value", "p", "end"], inplace=True)
gene_name_merge

C:\Users\39379\AppData\Local\Temp\ipykernel_7732\4090396922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_name_merge.drop(columns=["p_value", "p", "end"], inplace=True)


,chromosome_name,TSS,strand,gene_name,gene_stable_id,gene_id,gene_type
0,chr19,58858938,-,A1BG,ENSG00000121410,ENSG00000121410.12,protein_coding
1,chr10,52645379,-,A1CF,ENSG00000148584,ENSG00000148584.15,protein_coding
2,chr12,9268507,-,A2M,ENSG00000175899,ENSG00000175899.15,protein_coding
3,chr12,8975206,+,A2ML1,ENSG00000166535,ENSG00000166535.20,protein_coding
4,chr22,43116867,-,A4GALT,ENSG00000128274,ENSG00000128274.17,protein_coding
...,...,...,...,...,...,...,...
15615,chr1,53308425,+,ZYG11A,ENSG00000203995,ENSG00000203995.10,protein_coding
15616,chr1,53192114,+,ZYG11B,ENSG00000162378,ENSG00000162378.13,protein_coding
15617,chr7,143078379,+,ZYX,ENSG00000159840,ENSG00000159840.16,protein_coding
15618,chr17,4046257,-,ZZEF1,ENSG00000074755,ENSG00000074755.15,protein_coding


In [78]:
#metti gene_name_stable e gene_name come prime due colonne
cols = gene_name_merge.columns.tolist()
cols

['chromosome_name',
 'TSS',
 'strand',
 'gene_name',
 'gene_stable_id',
 'gene_id',
 'gene_type']

In [83]:
#riordina le colonne in modo da avere gene_name_stable e gene_name come prime due colonne
cols = cols[-2:] + cols[:-2]
cols

['gene_stable_id',
 'gene_id',
 'gene_type',
 'chromosome_name',
 'TSS',
 'strand',
 'gene_name']

In [89]:
gene_name_merge = gene_name_merge[cols]
gene_name_merge

,gene_stable_id,gene_id,gene_type,chromosome_name,TSS,strand,gene_name
0,ENSG00000121410,ENSG00000121410.12,protein_coding,chr19,58858938,-,A1BG
1,ENSG00000148584,ENSG00000148584.15,protein_coding,chr10,52645379,-,A1CF
2,ENSG00000175899,ENSG00000175899.15,protein_coding,chr12,9268507,-,A2M
3,ENSG00000166535,ENSG00000166535.20,protein_coding,chr12,8975206,+,A2ML1
4,ENSG00000128274,ENSG00000128274.17,protein_coding,chr22,43116867,-,A4GALT
...,...,...,...,...,...,...,...
15615,ENSG00000203995,ENSG00000203995.10,protein_coding,chr1,53308425,+,ZYG11A
15616,ENSG00000162378,ENSG00000162378.13,protein_coding,chr1,53192114,+,ZYG11B
15617,ENSG00000159840,ENSG00000159840.16,protein_coding,chr7,143078379,+,ZYX
15618,ENSG00000074755,ENSG00000074755.15,protein_coding,chr17,4046257,-,ZZEF1


In [94]:
# gene_name_merge.set_index("gene_stable_id", inplace=True)
#leva label da index
gene_name_merge.index.name=None
gene_name_merge


,gene_id,gene_type,chromosome_name,TSS,strand,gene_name
ENSG00000121410,ENSG00000121410.12,protein_coding,chr19,58858938,-,A1BG
ENSG00000148584,ENSG00000148584.15,protein_coding,chr10,52645379,-,A1CF
ENSG00000175899,ENSG00000175899.15,protein_coding,chr12,9268507,-,A2M
ENSG00000166535,ENSG00000166535.20,protein_coding,chr12,8975206,+,A2ML1
ENSG00000128274,ENSG00000128274.17,protein_coding,chr22,43116867,-,A4GALT
...,...,...,...,...,...,...
ENSG00000203995,ENSG00000203995.10,protein_coding,chr1,53308425,+,ZYG11A
ENSG00000162378,ENSG00000162378.13,protein_coding,chr1,53192114,+,ZYG11B
ENSG00000159840,ENSG00000159840.16,protein_coding,chr7,143078379,+,ZYX
ENSG00000074755,ENSG00000074755.15,protein_coding,chr17,4046257,-,ZZEF1


In [95]:
gene_name_merge.to_csv("..\\dataset\\df_alberto_tss_correct", sep="\t")

## salvo i 15k geni con i tss del bed, poi riordinerò le colonne come quelle di gx bert

In [86]:
df = pd.read_csv("..\\dataset\\df_alberto_tss_correct", sep="\t", index_col=0)
df

,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name
ENSG00000121410,A1BG,ENSG00000121410.12,protein_coding,58858938,-,chr19
ENSG00000148584,A1CF,ENSG00000148584.15,protein_coding,52645379,-,chr10
ENSG00000175899,A2M,ENSG00000175899.15,protein_coding,9268507,-,chr12
ENSG00000166535,A2ML1,ENSG00000166535.20,protein_coding,8975206,+,chr12
ENSG00000128274,A4GALT,ENSG00000128274.17,protein_coding,43116867,-,chr22
...,...,...,...,...,...,...
ENSG00000203995,ZYG11A,ENSG00000203995.10,protein_coding,53308425,+,chr1
ENSG00000162378,ZYG11B,ENSG00000162378.13,protein_coding,53192114,+,chr1
ENSG00000159840,ZYX,ENSG00000159840.16,protein_coding,143078379,+,chr7
ENSG00000074755,ZZEF1,ENSG00000074755.15,protein_coding,4046257,-,chr17


# gx_bert 

In [111]:
!pip install --upgrade gdown

  Using cached gdown-5.1.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   ---------------- ---------------------- 61.4/147.9 kB 656.4 kB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 1.1 MB/s eta 0:00:00


In [12]:
!gdown --folder https://drive.google.com/drive/folders/1m0FG0Jp30C69ldQpeV2znD1sdZHA0Nni?usp=share_link

Processing file 1avtBtLVT4DmMRqhKwZtL0NbEmijLOvbq CTB_128k_slack_0.h5
Processing file 1--K2OdkUqqsySACJVtNw1ZFom8i6s9qy CTB_128k_slack_1.h5
Processing file 1-3_DwYPWp8StDOdjj213XSihi8XUOlD4 CTB_128k_slack_2.h5
Processing file 1-6BLu2NayUc-eLMGdg790T-5R4QbeSpA CTB_128k_slack_3.h5
Processing file 1-ALMlaTjBIZuDQ1Y2scLAykN7zH-mP3u CTB_128k_slack_4.h5
Processing file 1-Drv_KwU9UDQ2V5TQuHtfvS1sYTe4heL CTB_128k_slack_5.h5
Processing file 1-JmxSnPt5AtFiBX82qWIBwX_s7BqiN2A CTB_128k_slack_6.h5


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1avtBtLVT4DmMRqhKwZtL0NbEmijLOvbq
From (redirected): https://drive.google.com/uc?id=1avtBtLVT4DmMRqhKwZtL0NbEmijLOvbq&confirm=t&uuid=c6d8fc38-e2b3-461d-a270-460e5ba43802
To: i:\Il mio Drive\UNImore\Secondo anno\AI for Bioinformatic\GX_alBERTo\data_manipulation\CTB\CTB_128k_slack_0.h5

  0%|          | 0.00/330M [00:00<?, ?B/s]
  0%|          | 524k/330M [00:00<07:05, 773kB/s]
  0%|          | 1.05M/330M [00:00<04:15, 1.28MB/s]
  0%|          | 1.57M/330M [00:01<03:33, 1.54MB/s]
  1%|          | 2.10M/330M [00:01<03:02, 1.79MB/s]
  1%|          | 2.62M/330M [00:01<03:25, 1.59MB/s]
  1%|          | 3.15M/330M [00:02<03:23, 1.60MB/s]
  1%|          | 3.67M/330M [00:02<03:14, 1.67MB/s]
  1%|▏         | 4.19M/330M [00:02<03:11, 1.70MB/s]
  1%|▏         | 4.72M/330M [00:02<03:10, 1.70MB/s]
  2%|▏    

In [95]:

newDataset1_0 = pd.read_hdf("../CTB/CTB_128k_slack_0.h5", key="df", mode="r")
newDataset1_1 = pd.read_hdf("../CTB/CTB_128k_slack_1.h5", key="df", mode="r")
newDataset1_2 = pd.read_hdf("../CTB/CTB_128k_slack_2.h5", key="df", mode="r")
newDataset1_3 = pd.read_hdf("../CTB/CTB_128k_slack_3.h5", key="df", mode="r")
newDataset1_4 = pd.read_hdf("../CTB/CTB_128k_slack_4.h5", key="df", mode="r")
newDataset1_5 = pd.read_hdf("../CTB/CTB_128k_slack_5.h5", key="df", mode="r")
newDataset1_6 = pd.read_hdf("../CTB/CTB_128k_slack_6.h5", key="df", mode="r")

dataset = pd.concat([
                            newDataset1_0,
                            newDataset1_1,
                            newDataset1_2,
                            newDataset1_3,
                            newDataset1_4,
                            newDataset1_5,
                            newDataset1_6,
                         ])
dataset



,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name,sequence,split,labels
ENSG00000121410,A1BG,ENSG00000121410.12,protein_coding,58347673,-,chr19,"[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, ...",train,0.128766
ENSG00000148584,A1CF,ENSG00000148584.15,protein_coding,50885633,-,chr10,"[1, 1, 3, 0, 0, 0, 3, 2, 2, 1, 1, 1, 2, 1, 2, ...",train,-1.442332
ENSG00000175899,A2M,ENSG00000175899.15,protein_coding,9115927,-,chr12,"[1, 2, 0, 3, 1, 1, 0, 1, 2, 1, 2, 2, 1, 1, 0, ...",train,1.172033
ENSG00000166535,A2ML1,ENSG00000166535.20,protein_coding,8822627,+,chr12,"[3, 1, 3, 0, 3, 2, 2, 0, 2, 2, 3, 1, 3, 2, 2, ...",train,-1.300226
ENSG00000128274,A4GALT,ENSG00000128274.17,protein_coding,42720879,-,chr22,"[1, 0, 1, 1, 2, 0, 3, 1, 0, 1, 3, 0, 1, 3, 0, ...",train,0.157178
...,...,...,...,...,...,...,...,...,...
ENSG00000166160,OPN1MW2,ENSG00000166160.9,protein_coding,154219756,+,chrX,"[0, 3, 3, 0, 3, 2, 1, 3, 3, 3, 2, 2, 1, 0, 2, ...",train,-1.463212
ENSG00000184033,CTAG1B,ENSG00000184033.14,protein_coding,154619282,-,chrX,"[3, 0, 1, 0, 3, 0, 0, 1, 0, 0, 3, 0, 0, 3, 3, ...",train,-1.463212
ENSG00000126890,CTAG2,ENSG00000126890.14,protein_coding,154653579,-,chrX,"[1, 2, 1, 3, 2, 0, 2, 0, 3, 1, 3, 1, 2, 2, 0, ...",train,-1.463212
ENSG00000203870,SMIM9,ENSG00000203870.5,protein_coding,154834662,-,chrX,"[0, 3, 3, 0, 3, 3, 1, 3, 3, 0, 3, 3, 1, 1, 3, ...",train,-1.463212


In [97]:
#droppa le ultime 3 colonne
dataset.drop(columns=[
                      "sequence",
                      "split",
                      "labels"
                      ],
                      inplace=True)


In [98]:
dataset

,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name
ENSG00000121410,A1BG,ENSG00000121410.12,protein_coding,58347673,-,chr19
ENSG00000148584,A1CF,ENSG00000148584.15,protein_coding,50885633,-,chr10
ENSG00000175899,A2M,ENSG00000175899.15,protein_coding,9115927,-,chr12
ENSG00000166535,A2ML1,ENSG00000166535.20,protein_coding,8822627,+,chr12
ENSG00000128274,A4GALT,ENSG00000128274.17,protein_coding,42720879,-,chr22
...,...,...,...,...,...,...
ENSG00000166160,OPN1MW2,ENSG00000166160.9,protein_coding,154219756,+,chrX
ENSG00000184033,CTAG1B,ENSG00000184033.14,protein_coding,154619282,-,chrX
ENSG00000126890,CTAG2,ENSG00000126890.14,protein_coding,154653579,-,chrX
ENSG00000203870,SMIM9,ENSG00000203870.5,protein_coding,154834662,-,chrX


In [10]:
df.head()

,gene_id,gene_type,chromosome_name,TSS,strand,gene_name
ENSG00000121410,ENSG00000121410.12,protein_coding,chr19,58858938,-,A1BG
ENSG00000148584,ENSG00000148584.15,protein_coding,chr10,52645379,-,A1CF
ENSG00000175899,ENSG00000175899.15,protein_coding,chr12,9268507,-,A2M
ENSG00000166535,ENSG00000166535.20,protein_coding,chr12,8975206,+,A2ML1
ENSG00000128274,ENSG00000128274.17,protein_coding,chr22,43116867,-,A4GALT


In [ ]:
#rename in df strand as genomic_strand
df.rename(columns={"strand":"genomic_strand"}, inplace=True)

In [7]:
cols = dataset.columns.to_list()

In [14]:
df[cols]

,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name
ENSG00000121410,A1BG,ENSG00000121410.12,protein_coding,58858938,-,chr19
ENSG00000148584,A1CF,ENSG00000148584.15,protein_coding,52645379,-,chr10
ENSG00000175899,A2M,ENSG00000175899.15,protein_coding,9268507,-,chr12
ENSG00000166535,A2ML1,ENSG00000166535.20,protein_coding,8975206,+,chr12
ENSG00000128274,A4GALT,ENSG00000128274.17,protein_coding,43116867,-,chr22
...,...,...,...,...,...,...
ENSG00000203995,ZYG11A,ENSG00000203995.10,protein_coding,53308425,+,chr1
ENSG00000162378,ZYG11B,ENSG00000162378.13,protein_coding,53192114,+,chr1
ENSG00000159840,ZYX,ENSG00000159840.16,protein_coding,143078379,+,chr7
ENSG00000074755,ZZEF1,ENSG00000074755.15,protein_coding,4046257,-,chr17


## rinomino il df df_alberto_tss_correct con l'ordine delle colonne di gx bert

# aggiungo label al dataset con i tss corretti
- se uso 
    - label = pd.read_csv("..\\dataset\\df_alBERTo.csv")
        - allora aggiungo label usando ENSG00000203995 (senza punto)

- altrimenti con:
    - label = pd.read_csv("..\\dataset\\Dataset_median.csv")
        - allora aggiungo label usando ENSG00000203995.8 (con punto) 
- nel caso con punto perdo più geni ottenendo 14k geni invece di ~ 15k 


In [60]:
# usa daataset_median o df_alberto in base a se si vuole fare merge su gene_id con o senza punto

label = pd.read_csv("..\\dataset\\Dataset_median.csv")

In [87]:
label = pd.read_csv("..\\dataset\\df_alBERTo.csv")
label = label[["gene_id","fpkm_median","tpm_median","fpkm_uq_median"]]

In [88]:
label

,gene_id,fpkm_median,tpm_median,fpkm_uq_median
0,ENSG00000121410,-4.563199,-3.172669,-4.445362
1,ENSG00000148584,-7.243318,-6.687800,-7.179188
2,ENSG00000175899,5.214346,6.680884,5.391613
3,ENSG00000166535,-2.501443,-1.024150,-2.346665
4,ENSG00000128274,2.025064,3.517855,2.194465
...,...,...,...,...
18123,ENSG00000259120,-1.905088,-0.462754,-1.762192
18124,ENSG00000259224,-3.590745,-2.099742,-3.461164
18125,ENSG00000259305,-3.097888,-1.617056,-2.919206
18126,ENSG00000259399,-7.643856,-7.643856,-7.643856


In [90]:
#togli da df la roba dopo il punto per la colonna gene_id
df['gene_id'] = df['gene_id'].apply(lambda x: x.split(".")[0])
df

,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name
ENSG00000121410,A1BG,ENSG00000121410,protein_coding,58858938,-,chr19
ENSG00000148584,A1CF,ENSG00000148584,protein_coding,52645379,-,chr10
ENSG00000175899,A2M,ENSG00000175899,protein_coding,9268507,-,chr12
ENSG00000166535,A2ML1,ENSG00000166535,protein_coding,8975206,+,chr12
ENSG00000128274,A4GALT,ENSG00000128274,protein_coding,43116867,-,chr22
...,...,...,...,...,...,...
ENSG00000203995,ZYG11A,ENSG00000203995,protein_coding,53308425,+,chr1
ENSG00000162378,ZYG11B,ENSG00000162378,protein_coding,53192114,+,chr1
ENSG00000159840,ZYX,ENSG00000159840,protein_coding,143078379,+,chr7
ENSG00000074755,ZZEF1,ENSG00000074755,protein_coding,4046257,-,chr17


In [91]:
# merge tra df e label su gene_id mantenendo come index l'index di df
df_label = pd.merge(df, label , on="gene_id")
df_label


,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name,fpkm_median,tpm_median,fpkm_uq_median
0,A1BG,ENSG00000121410,protein_coding,58858938,-,chr19,-4.563199,-3.172669,-4.445362
1,A1CF,ENSG00000148584,protein_coding,52645379,-,chr10,-7.243318,-6.687800,-7.179188
2,A2M,ENSG00000175899,protein_coding,9268507,-,chr12,5.214346,6.680884,5.391613
3,A2ML1,ENSG00000166535,protein_coding,8975206,+,chr12,-2.501443,-1.024150,-2.346665
4,A4GALT,ENSG00000128274,protein_coding,43116867,-,chr22,2.025064,3.517855,2.194465
...,...,...,...,...,...,...,...,...,...
15579,ZYG11A,ENSG00000203995,protein_coding,53308425,+,chr1,-1.827256,-0.404114,-1.655626
15580,ZYG11B,ENSG00000162378,protein_coding,53192114,+,chr1,2.230572,3.640471,2.432452
15581,ZYX,ENSG00000159840,protein_coding,143078379,+,chr7,6.632143,8.096538,6.811657
15582,ZZEF1,ENSG00000074755,protein_coding,4046257,-,chr17,1.151924,2.662821,1.366308


## salvataggio vari dataset con != numero di geni che sto creando

In [93]:
# df_label.to_csv("..\\dataset\\df_alberto_tss_correct_label.csv", index=False)
df_label.to_csv("..\\dataset\\df_alberto_tss_correct_label_16k.csv", index=False)

In [85]:
df_label = pd.read_csv("..\\dataset\\df_alberto_tss_correct_label.csv", index_col=0)
df_label

,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name,fpkm_median,tpm_median,fpkm_uq_median
gene,,,,,,,,,
ENSG00000121410,A1BG,ENSG00000121410.12,protein_coding,58858938,-,chr19,-4.563199,-3.172669,-4.445362
ENSG00000148584,A1CF,ENSG00000148584.15,protein_coding,52645379,-,chr10,-7.243318,-6.687800,-7.179188
ENSG00000175899,A2M,ENSG00000175899.15,protein_coding,9268507,-,chr12,5.214346,6.680884,5.391613
ENSG00000166535,A2ML1,ENSG00000166535.20,protein_coding,8975206,+,chr12,-2.501443,-1.024150,-2.346665
ENSG00000128274,A4GALT,ENSG00000128274.17,protein_coding,43116867,-,chr22,2.025064,3.517855,2.194465
...,...,...,...,...,...,...,...,...,...
ENSG00000203995,ZYG11A,ENSG00000203995.10,protein_coding,53308425,+,chr1,-1.827256,-0.404114,-1.655626
ENSG00000162378,ZYG11B,ENSG00000162378.13,protein_coding,53192114,+,chr1,2.230572,3.640471,2.432452
ENSG00000159840,ZYX,ENSG00000159840.16,protein_coding,143078379,+,chr7,6.632143,8.096538,6.811657


## prendo da alberto i geni che mancano in tss_correct e li aggiungo al dataset da ~16k geni per arrivare a ~18k

In [96]:
alberto = pd.read_csv("..\\dataset\\df_alBERTo.csv")
alberto 

,gene_id,gene_type,split,labels,gene_name,TSS,chromosome_name,strand,fpkm_median,tpm_median,fpkm_uq_median
0,ENSG00000121410,protein_coding,train,0.128766,A1BG,58347673,chr19,-,-4.563199,-3.172669,-4.445362
1,ENSG00000148584,protein_coding,train,-1.442332,A1CF,50885633,chr10,-,-7.243318,-6.687800,-7.179188
2,ENSG00000175899,protein_coding,train,1.172033,A2M,9115927,chr12,-,5.214346,6.680884,5.391613
3,ENSG00000166535,protein_coding,train,-1.300226,A2ML1,8822627,chr12,+,-2.501443,-1.024150,-2.346665
4,ENSG00000128274,protein_coding,train,0.157178,A4GALT,42720879,chr22,-,2.025064,3.517855,2.194465
...,...,...,...,...,...,...,...,...,...,...,...
18123,ENSG00000259120,protein_coding,test,-1.050895,SMIM6,73642517,chr17,+,-1.905088,-0.462754,-1.762192
18124,ENSG00000259224,protein_coding,train,-1.392807,SLC35G6,7384721,chr17,+,-3.590745,-2.099742,-3.461164
18125,ENSG00000259305,protein_coding,train,0.848319,ZHX1-C8orf76,124279627,chr8,-,-3.097888,-1.617056,-2.919206
18126,ENSG00000259399,protein_coding,train,1.680397,TGIF2-RAB5IF,35202956,chr20,+,-7.643856,-7.643856,-7.643856


In [101]:
#prendi tutti i gene_id in dataset che non sono in df_alberto_tss_correct
geni_mancanti = alberto[~alberto.gene_id.isin(df.gene_id)]
# geni_mancanti =geni_mancanti['gene_id']


,gene_id,gene_type,split,labels,gene_name,TSS,chromosome_name,strand,fpkm_median,tpm_median,fpkm_uq_median
74,ENSG00000114786,protein_coding,train,0.955634,ABHD14A-ACY1,51974055,chr3,+,-5.368849,-4.115285,-5.216250
608,ENSG00000198483,protein_coding,train,-0.427562,ANKRD35,145872779,chr1,-,-1.996542,-0.622154,-1.827256
609,ENSG00000174501,protein_coding,train,-0.043570,ANKRD36C,97113660,chr2,-,-1.434403,-0.003611,-1.233786
1150,ENSG00000198049,protein_coding,train,-1.435572,AVPR1B,206117389,chr1,-,-2.807932,-1.388733,-2.651994
1963,ENSG00000147996,protein_coding,train,-0.113827,CBWD5,65675958,chr9,+,-1.127526,0.325156,-0.920025
...,...,...,...,...,...,...,...,...,...,...,...
18123,ENSG00000259120,protein_coding,test,-1.050895,SMIM6,73642517,chr17,+,-1.905088,-0.462754,-1.762192
18124,ENSG00000259224,protein_coding,train,-1.392807,SLC35G6,7384721,chr17,+,-3.590745,-2.099742,-3.461164
18125,ENSG00000259305,protein_coding,train,0.848319,ZHX1-C8orf76,124279627,chr8,-,-3.097888,-1.617056,-2.919206
18126,ENSG00000259399,protein_coding,train,1.680397,TGIF2-RAB5IF,35202956,chr20,+,-7.643856,-7.643856,-7.643856


In [102]:
geni_mancanti.drop(columns=["split"], inplace=True)

C:\Users\39379\AppData\Local\Temp\ipykernel_13764\3649069889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geni_mancanti.drop(columns=["split"], inplace=True)


In [114]:
# Rinomina le colonne in geni_mancanti per corrispondere a quelle di df_label
geni_mancanti = geni_mancanti.rename(columns={'strand': 'genomic_strand'})

geni_mancanti.columns.to_list(), len(geni_mancanti.columns.to_list())



(['gene_id',
  'gene_type',
  'labels',
  'gene_name',
  'TSS',
  'chromosome_name',
  'genomic_strand',
  'fpkm_median',
  'tpm_median',
  'fpkm_uq_median'],
 10)

In [110]:
cols = df_label.columns.to_list()
cols, len(cols)


(['gene_name',
  'gene_id',
  'gene_type',
  'TSS',
  'genomic_strand',
  'chromosome_name',
  'fpkm_median',
  'tpm_median',
  'fpkm_uq_median'],
 9)

In [115]:
#riordino colonne di geni_mancanti in modo da avere le stesse colonne di df_label
geni_mancanti = geni_mancanti[cols]

In [116]:
geni_mancanti



,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name,fpkm_median,tpm_median,fpkm_uq_median
74,ABHD14A-ACY1,ENSG00000114786,protein_coding,51974055,+,chr3,-5.368849,-4.115285,-5.216250
608,ANKRD35,ENSG00000198483,protein_coding,145872779,-,chr1,-1.996542,-0.622154,-1.827256
609,ANKRD36C,ENSG00000174501,protein_coding,97113660,-,chr2,-1.434403,-0.003611,-1.233786
1150,AVPR1B,ENSG00000198049,protein_coding,206117389,-,chr1,-2.807932,-1.388733,-2.651994
1963,CBWD5,ENSG00000147996,protein_coding,65675958,+,chr9,-1.127526,0.325156,-0.920025
...,...,...,...,...,...,...,...,...,...
18123,SMIM6,ENSG00000259120,protein_coding,73642517,+,chr17,-1.905088,-0.462754,-1.762192
18124,SLC35G6,ENSG00000259224,protein_coding,7384721,+,chr17,-3.590745,-2.099742,-3.461164
18125,ZHX1-C8orf76,ENSG00000259305,protein_coding,124279627,-,chr8,-3.097888,-1.617056,-2.919206
18126,TGIF2-RAB5IF,ENSG00000259399,protein_coding,35202956,+,chr20,-7.643856,-7.643856,-7.643856


In [117]:
#concatena df_label e geni_mancanti
df_totale = pd.concat([df_label, geni_mancanti])
df_totale

,gene_name,gene_id,gene_type,TSS,genomic_strand,chromosome_name,fpkm_median,tpm_median,fpkm_uq_median
0,A1BG,ENSG00000121410,protein_coding,58858938,-,chr19,-4.563199,-3.172669,-4.445362
1,A1CF,ENSG00000148584,protein_coding,52645379,-,chr10,-7.243318,-6.687800,-7.179188
2,A2M,ENSG00000175899,protein_coding,9268507,-,chr12,5.214346,6.680884,5.391613
3,A2ML1,ENSG00000166535,protein_coding,8975206,+,chr12,-2.501443,-1.024150,-2.346665
4,A4GALT,ENSG00000128274,protein_coding,43116867,-,chr22,2.025064,3.517855,2.194465
...,...,...,...,...,...,...,...,...,...
18123,SMIM6,ENSG00000259120,protein_coding,73642517,+,chr17,-1.905088,-0.462754,-1.762192
18124,SLC35G6,ENSG00000259224,protein_coding,7384721,+,chr17,-3.590745,-2.099742,-3.461164
18125,ZHX1-C8orf76,ENSG00000259305,protein_coding,124279627,-,chr8,-3.097888,-1.617056,-2.919206
18126,TGIF2-RAB5IF,ENSG00000259399,protein_coding,35202956,+,chr20,-7.643856,-7.643856,-7.643856


In [ ]:
df_totale.to_csv("..\\dataset\\df_alberto_tss_correct_label_18k.csv", index=False)

In [20]:
# meging=pd.read_csv("..\\dataset\\ens37_mancanti.txt", sep="\t")
meging=pd.read_csv("..\\dataset\\df_alBERTo.csv")
meging




,gene_id,gene_type,split,labels,gene_name,TSS,chromosome_name,strand,fpkm_median,tpm_median,fpkm_uq_median
0,ENSG00000121410,protein_coding,train,0.128766,A1BG,58347673,chr19,-,-4.563199,-3.172669,-4.445362
1,ENSG00000148584,protein_coding,train,-1.442332,A1CF,50885633,chr10,-,-7.243318,-6.687800,-7.179188
2,ENSG00000175899,protein_coding,train,1.172033,A2M,9115927,chr12,-,5.214346,6.680884,5.391613
3,ENSG00000166535,protein_coding,train,-1.300226,A2ML1,8822627,chr12,+,-2.501443,-1.024150,-2.346665
4,ENSG00000128274,protein_coding,train,0.157178,A4GALT,42720879,chr22,-,2.025064,3.517855,2.194465
...,...,...,...,...,...,...,...,...,...,...,...
18123,ENSG00000259120,protein_coding,test,-1.050895,SMIM6,73642517,chr17,+,-1.905088,-0.462754,-1.762192
18124,ENSG00000259224,protein_coding,train,-1.392807,SLC35G6,7384721,chr17,+,-3.590745,-2.099742,-3.461164
18125,ENSG00000259305,protein_coding,train,0.848319,ZHX1-C8orf76,124279627,chr8,-,-3.097888,-1.617056,-2.919206
18126,ENSG00000259399,protein_coding,train,1.680397,TGIF2-RAB5IF,35202956,chr20,+,-7.643856,-7.643856,-7.643856


In [27]:
meging.gene_id.isin(geni_mancanti.gene_id).value_counts()

# geni_mancanti.gene_id.isin(meging.gene_id)

gene_id
False    15584
True      2544
Name: count, dtype: int64

In [28]:
#seleziona i gene_id in meging che sono in df3
geni_mancanti_final = meging[meging.gene_id.isin(geni_mancanti.gene_id)]


,gene_id,gene_type,split,labels,gene_name,TSS,chromosome_name,strand,fpkm_median,tpm_median,fpkm_uq_median
74,ENSG00000114786,protein_coding,train,0.955634,ABHD14A-ACY1,51974055,chr3,+,-5.368849,-4.115285,-5.216250
608,ENSG00000198483,protein_coding,train,-0.427562,ANKRD35,145872779,chr1,-,-1.996542,-0.622154,-1.827256
609,ENSG00000174501,protein_coding,train,-0.043570,ANKRD36C,97113660,chr2,-,-1.434403,-0.003611,-1.233786
1150,ENSG00000198049,protein_coding,train,-1.435572,AVPR1B,206117389,chr1,-,-2.807932,-1.388733,-2.651994
1963,ENSG00000147996,protein_coding,train,-0.113827,CBWD5,65675958,chr9,+,-1.127526,0.325156,-0.920025
...,...,...,...,...,...,...,...,...,...,...,...
18123,ENSG00000259120,protein_coding,test,-1.050895,SMIM6,73642517,chr17,+,-1.905088,-0.462754,-1.762192
18124,ENSG00000259224,protein_coding,train,-1.392807,SLC35G6,7384721,chr17,+,-3.590745,-2.099742,-3.461164
18125,ENSG00000259305,protein_coding,train,0.848319,ZHX1-C8orf76,124279627,chr8,-,-3.097888,-1.617056,-2.919206
18126,ENSG00000259399,protein_coding,train,1.680397,TGIF2-RAB5IF,35202956,chr20,+,-7.643856,-7.643856,-7.643856


In [53]:
geni_mancanti_final.to_csv("..\\dataset\\geni_mancanti_final.txt", sep="\t")

In [42]:
# droppa da gene_id di label il contenuto dopo il punto
label['gene_id'] = label['gene_id'].apply(lambda x: x.split(".")[0])

In [43]:
label

,gene_id,fpkm_median,tpm_median,fpkm_uq_median
0,ENSG00000000003,4.062985,5.543366,4.292598
1,ENSG00000000005,-3.925769,-2.475535,-3.733124
2,ENSG00000000419,5.477703,6.963690,5.659445
3,ENSG00000000457,0.873892,2.396543,1.063089
4,ENSG00000000460,0.652142,2.088786,0.803392
...,...,...,...,...
60655,ENSG00000288669,-7.643856,-7.643856,-7.643856
60656,ENSG00000288670,1.319098,2.787955,1.484344
60657,ENSG00000288671,-7.643856,-7.643856,-7.643856
60658,ENSG00000288674,-5.429731,-4.173970,-5.293359


In [44]:
#merge di geni_mancanti_final e label su gene_id
geni_mancanti_label = pd.merge(geni_mancanti_final, label, on="gene_id")
geni_mancanti_label


,gene_id,gene_type,split,labels,gene_name,TSS,chromosome_name,strand,fpkm_median_x,tpm_median_x,fpkm_uq_median_x,fpkm_median_y,tpm_median_y,fpkm_uq_median_y
0,ENSG00000114786,protein_coding,train,0.955634,ABHD14A-ACY1,51974055,chr3,+,-5.368849,-4.115285,-5.216250,-5.368849,-4.115285,-5.216250
1,ENSG00000198483,protein_coding,train,-0.427562,ANKRD35,145872779,chr1,-,-1.996542,-0.622154,-1.827256,-1.996542,-0.622154,-1.827256
2,ENSG00000174501,protein_coding,train,-0.043570,ANKRD36C,97113660,chr2,-,-1.434403,-0.003611,-1.233786,-1.434403,-0.003611,-1.233786
3,ENSG00000198049,protein_coding,train,-1.435572,AVPR1B,206117389,chr1,-,-2.807932,-1.388733,-2.651994,-2.807932,-1.388733,-2.651994
4,ENSG00000147996,protein_coding,train,-0.113827,CBWD5,65675958,chr9,+,-1.127526,0.325156,-0.920025,-1.127526,0.325156,-0.920025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,ENSG00000259120,protein_coding,test,-1.050895,SMIM6,73642517,chr17,+,-1.905088,-0.462754,-1.762192,-1.905088,-0.462754,-1.762192
2541,ENSG00000259224,protein_coding,train,-1.392807,SLC35G6,7384721,chr17,+,-3.590745,-2.099742,-3.461164,-3.590745,-2.099742,-3.461164
2542,ENSG00000259305,protein_coding,train,0.848319,ZHX1-C8orf76,124279627,chr8,-,-3.097888,-1.617056,-2.919206,-3.097888,-1.617056,-2.919206
2543,ENSG00000259399,protein_coding,train,1.680397,TGIF2-RAB5IF,35202956,chr20,+,-7.643856,-7.643856,-7.643856,-7.643856,-7.643856,-7.643856


# metti le label come sotto da dataset median per fa le sequnze 

In [54]:
geni_mancanti_label.to_csv("..\\dataset\\geni_mancanti_label.txt", sep="\t")

In [4]:
patient=pd.read_csv("..\\dataset\\Dataset_median.csv", sep=",")
#merge su gene_id
df2 = pd.merge(df1, patient, on="gene_id")
df2

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id,fpkm_median,tpm_median,fpkm_uq_median
0,chr10,100015362,100015397,"p2@LOXL4,0.1291",55.0,-,100015362,100015397,"211,211,211",p2,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
1,chr10,100017518,100017519,"p3@LOXL4,0.1842",13.0,-,100017518,100017519,"30,144,255",p3,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
2,chr10,100027943,100027958,"p1@LOXL4,0.2200",48.0,-,100027943,100027958,"30,144,255",p1,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
3,chr10,100174900,100174956,"p1@PYROXD2,0.2721",0.0,-,100174900,100174956,"60,179,113",p1,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999
4,chr10,100174957,100174982,"p2@PYROXD2,0.2448",0.0,-,100174957,100174982,"60,179,113",p2,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153925,chrY,2870826,2870844,"p2@ZFY-AS1,0.0950",-159.0,-,2870826,2870844,"211,211,211",p2,ZFY-AS1,ENSG00000233070.1,-7.643856,-7.643856,-7.643856
153926,chrY,6778505,6778534,"p6@TBL1Y,0.3422",-192.0,+,6778505,6778534,"60,179,113",p6,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856
153927,chrY,6778592,6778601,"p7@TBL1Y,0.4173",-125.0,+,6778592,6778601,"60,179,113",p7,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856
153928,chrY,6778602,6778616,"p4@TBL1Y,0.4601",-110.0,+,6778602,6778616,"60,179,113",p4,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856


NameError: name 'df2' is not defined

In [55]:
df2.to_csv('df_alberto_tss_correct_label.csv', index=False)

NameError: name 'df2' is not defined

In [9]:
df2["gene_name"].value_counts()


gene_name
MEG3         176
SLC1A2       170
PAPPA        153
APOB         147
QKI          137
            ... 
PAPPA-AS1      1
MRPL50P2       1
RPS26P49       1
KRT8P15        1
ANKRD36P1      1
Name: count, Length: 14198, dtype: int64

In [28]:

df3 = df2.loc[df2.groupby("gene_name")["p_value"].idxmin()]



df3.value_counts("p")

p
p1    14075
p2       86
p3       14
p4       10
p5        5
p6        3
p7        2
p8        2
p9        1
Name: count, dtype: int64

In [27]:
df4 = df2.loc[df2.groupby("gene_id")["p_value"].idxmin()]
df4

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id,fpkm_median,tpm_median,fpkm_uq_median,p_value
153866,chrX,99839925,99839938,"p1@TNMD,0.1353",126.0,+,99839925,99839938,"211,211,211",p1,TNMD,ENSG00000000005.6,-3.925769,-2.475535,-3.733124,1
38903,chr1,169863016,169863109,"p1@SCYL3,0.2639",0.0,-,169863016,169863109,"60,179,113",p1,SCYL3,ENSG00000000457.14,0.873892,2.396543,1.063089,1
42198,chr1,27961629,27961665,"p1@FGR,0.5037",0.0,-,27961629,27961665,"60,179,113",p1,FGR,ENSG00000000938.13,0.612022,2.081715,0.757621,1
71796,chr6,143832820,143832875,"p1@FUCA2,0.4000",0.0,-,143832820,143832875,"60,179,113",p1,FUCA2,ENSG00000001036.14,3.827545,5.333456,4.001946,1
74636,chr6,53409890,53409906,"p1@GCLC,0.4228",-4.0,-,53409890,53409906,"60,179,113",p1,GCLC,ENSG00000001084.13,1.509544,2.956224,1.694702,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86592,chrX,101967366,101967391,"p1@ARMCX5-GPRASP2,p1@GPRASP2,0.6351",0.0,+,101967366,101967391,"60,179,113",p1,ARMCX5-GPRASP2,ENSG00000286237.1,-7.643856,-7.643856,-7.643856,1
65961,chr4,89513578,89513670,"p1@HERC3,0.3732",0.0,+,89513578,89513670,"60,179,113",p1,HERC3,ENSG00000287542.1,-7.643856,-7.643856,-7.643856,1
82689,chr8,67579807,67579860,"p1@C8orf44,p1@SGK3,0.4647",0.0,+,67579807,67579860,"60,179,113",p1,C8orf44,ENSG00000288596.2,0.995448,2.458933,1.200316,1
148900,chr8,53852965,53852990,"p1@NPBWR1,0.2996",497.0,+,53852965,53852990,"60,179,113",p1,NPBWR1,ENSG00000288611.1,-6.687800,-5.779918,-6.546245,1
